In [4]:
from IPython.display import Image, display
import pickle
import os
import torch
from torch_geometric.utils.convert import from_networkx
import networkx as nx
import matplotlib.pyplot as plt
import graphviz
import multiprocessing as mp
from tqdm.contrib.concurrent import process_map  # or thread_map
import tqdm
import time
from parallelbar import progress_map
import numpy as np

In [5]:
def graph_from_dot(fpath):
    return nx.drawing.nx_pydot.read_dot(fpath)

In [6]:
pool = mp.Pool(10)

In [7]:
args_iterable = os.listdir("data/VDISC/test/joern/cfg/")
args_iterable = ["data/VDISC/test/joern/cfg/" + a for a in args_iterable]
cfg_graphs = list(tqdm.tqdm(pool.imap(graph_from_dot, args_iterable), total=len(args_iterable)))

100%|█████████████████████████████████████████████████| 529120/529120 [1:28:02<00:00, 100.16it/s]


In [9]:
file_names = pickle.load(open("test/file_names", "rb"))
func_names = pickle.load(open("test/func_names", "rb"))

In [11]:
def get_graph_indices_names(graphs):
    to_ret = {}
    for i in range(len(graphs)):
        G = graphs[i]
        pos = [n for n,d in G.in_degree() if d==0]
        name_full = G.nodes[pos[0]]['label']
        key = name_full[name_full.find(",") + 1:-2]
        to_ret[key] = i
    return to_ret

In [12]:
graph_names = get_graph_indices_names(cfg_graphs)

In [13]:
def find_graph(graph_names, graphs, file_name, func_name):
    label = 0
    if "vuln" in file_name:
        label = 1
    try:
        idx = graph_names[func_name.strip()]
        return (label, graphs[idx])
    except KeyError:
        print("Function name ''" + func_name + "' not found, skipping.")
        return ()

In [15]:
tuples = []
for i in range(len(file_names)):
    #print(func_names[i].strip())
    tuples.append(find_graph(graph_names, cfg_graphs, file_names[i], func_names[i]))

Function name ''common_select_close' not found, skipping.
Function name ''Exp_MatchMaxObjCmd' not found, skipping.
Function name ''gen_split_2110 ' not found, skipping.
Function name ''external_check_param_set' not found, skipping.
Function name ''iwl_dbgfs_scan_ant_rxchain_read' not found, skipping.
Function name ''pdo_sqlite_get_attribute' not found, skipping.
Function name ''upnp_dscv_got_external_ip' not found, skipping.
Function name ''mbox_test_message_write' not found, skipping.
Function name ''tile_state_notify ' not found, skipping.
Function name ''OnDelete' not found, skipping.
Function name ''pump_update_resume_path ' not found, skipping.
Function name ''__ecereMethod___ecereNameSpace__ecere__gui__controls__DataRow_Edit' not found, skipping.
Function name ''sftkdb_resolveConflicts' not found, skipping.
Function name ''getmax_box' not found, skipping.
Function name ''php_stream_open_for_zend_ex' not found, skipping.
Function name ''sst_module_new' not found, skipping.
Functio

In [16]:
pickle.dump(tuples, open("test/tuples_cfg.pkl", "wb"))

In [21]:
tuples[1][1].number_of_nodes()

35